In [21]:
import matplotlib.pyplot as plt
import numpy as np
from qutip import ( basis, expect, mesolve, qeye, sigmax, sigmay, sigmaz,destroy,
                   tensor)
from qutip.solver import heom
from scipy.integrate import quad
%matplotlib inline
# Set the system parameters
N = 3
m = 1e-3 # mass term
g=1
a=1
J = 1/4*a
theta=1
# initial state
state_list = [basis(2, 1)] + [basis(2, 0)] * (N - 1)
psi0 = tensor(state_list)

# Interaction coefficients
# Jx = 0.2 * np.pi * np.ones(N)
# Jy = 0.2 * np.pi * np.ones(N)
# Jz = 0.2 * np.pi * np.ones(N)

sx_list, sy_list, sz_list = [], [], []
for i in range(N):
    op_list = [qeye(2)] * N
    op_list[i] = sigmax()
    sx_list.append(tensor(op_list))
    op_list[i] = sigmay()
    sy_list.append(tensor(op_list))
    op_list[i] = sigmaz()
    sz_list.append(tensor(op_list))

# Hamiltonian - Energy splitting terms
H = 0
for i in range(N):
    H += (m/2)* (-1)**i * sz_list[i]

for n in range(N - 1):
    H += J * sx_list[n] * sx_list[n + 1]
    H += J * sy_list[n] * sy_list[n + 1]  
    
# Electromagnetic coupling term
for n in range(1,N):
    temp=0
    for i in range(1,n):
        temp += (sz_list[i] + (-1)**i )/2  + theta/(2*np.pi)
    H+=(temp**2 )*8*J *g**2

In [23]:
H

Quantum object: dims=[[2, 2, 2], [2, 2, 2]], shape=(8, 8), type='oper', isherm=True
Qobj data =
[[0.05116059 0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.05016059 0.5        0.         0.         0.
  0.         0.        ]
 [0.         0.5        1.41554082 0.         0.5        0.
  0.         0.        ]
 [0.         0.         0.         1.41454082 0.         0.5
  0.         0.        ]
 [0.         0.         0.5        0.         0.05016059 0.
  0.         0.        ]
 [0.         0.         0.         0.5        0.         0.04916059
  0.5        0.        ]
 [0.         0.         0.         0.         0.         0.5
  1.41454082 0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         1.41354082]]

In [1]:
def prepare_states(
        H, Q, E01,rho0, times=np.linspace(0, 25, 5000),
        Ni=1, Nr=2):
    # Raise this question about the paper / I consider this cheating
    gamma = E01/2
    w0 = 2*E01
    Gamma = gamma/2
    Omega = np.sqrt(w0**2 - Gamma**2 + 0j)
    lam = 1.15*np.sqrt(Omega)
    # functions don't take zero because 1/T should be fixed
    bath = heom.UnderDampedBath(
        Q=Q,
        lam=lam, gamma=gamma, w0=w0, T=0, Nk=5)
    cfiitter2 = heom.CorrelationFitter(
        Q, 0, times, bath.correlation_function)
    bath1, fit2info = cfiitter2.get_fit(Ni=Ni, Nr=Nr)
    print(fit2info['summary'])  # notice one mode is also a pretty good approximation
    times2=times[0:-1:10]
    cvis = bath.correlation_function(times2)
    plt.plot(times2, np.imag(cvis))
    plt.plot(times2, np.imag(bath1.correlation_function_approx(times2)), "-.")
    plt.show()
    solver = heom.HEOMSolver(H,
                             [bath1], max_depth=6, options={"atol": 1e-14})
    result = solver.run(rho0, times)
    cum = csolve(
        Hsys=H, t=times, baths=[bath],
        Qs=[Q],
        eps=1e-4, cython=False)


    a_ops = [[Q, bath.power_spectrum]]
    resultBR = qt.brmesolve(H, rho0, times, a_ops=a_ops, options={
                            "atol": 1e-14}, sec_cutoff=-1)

    a_ops = [[Q, bath.power_spectrum]]
    resultBR2 = qt.brmesolve(H, rho0, times, a_ops=a_ops, options={
        "atol": 1e-14})
    return result, cum,resultBR,resultBR2

NameError: name 'np' is not defined

In [24]:
# times=p.linspace(0, 25, 5000)
# plt.plot(times, sdd-np.min(H.eigenenergies()), label="Ohmic")
# plt.plot(times, sdd2-np.min(H.eigenenergies()), label="Underdamped")
# plt.plot(times, sdd3 - np.min(H.eigenenergies()),
#          label="Underdamped no cheating", linestyle="-.")
# plt.plot(times, sddbr-np.min(H.eigenenergies()), label="BR")
# plt.plot(times, sddcum-np.min(H.eigenenergies()),
#          label="CUM")  # plt.axhline(y=0)

# plt.yscale("log")
# plt.legend()